In [4]:
import pandas as pd
import pyarrow as pa
from pyarrow import parquet as pq
import glob
import time
import datetime as dt

In [5]:
parquet_files = glob.glob('data/*.parquet')

In [6]:
parquet_tables = []

In [7]:
for file in parquet_files:
        parquet_table = pq.read_table(file)
        parquet_tables.append(parquet_table)

print(f'We have data of {len(parquet_tables)} months')

We have data of 12 months


In [8]:
trips = pa.concat_tables(parquet_tables, ignore_index=True)

In [9]:
start_time = time.time()

In [10]:
trips = trips.to_pandas()

In [11]:
print(f'--- {time.time() - start_time} seconds ---')

--- 108.8759400844574 seconds ---


In [12]:
trips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39656098 entries, 0 to 39656097
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee         

In [13]:
start_time = time.time()

In [14]:
## This step is not necessary, there is just one duplicate and this step take something like 25 minutes. 
# Before: 39656098 entries
# After: 39656097 entries
## trips = trips.drop_duplicates().reset_index(drop=True) 
trips['trip_id'] = trips.index

In [15]:
print(f'--- {time.time() - start_time} seconds ---')

--- 7.014785051345825 seconds ---


In [16]:
trips.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,trip_id
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0,0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0,1
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0,2
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0,3
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0,4


In [17]:
trips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39656098 entries, 0 to 39656097
Data columns (total 20 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee         

In [ ]:
#trips.to_csv('data/yellowTrips.csv')

In [ ]:
trips['tpep_pickup_datetime'] = pd.to_datetime(trips['tpep_pickup_datetime'])
trips['tpep_dropoff_datetime'] = pd.to_datetime(trips['tpep_dropoff_datetime'])

In [ ]:
datetime_dim = trips[['tpep_pickup_datetime','tpep_dropoff_datetime']].reset_index(drop = True)

datetime_dim['datetime_id'] = datetime_dim.index

datetime_dim['pickup_time'] = datetime_dim['tpep_pickup_datetime'].dt.strftime('%H:%M:%S')
datetime_dim['pickup_day'] = datetime_dim['tpep_pickup_datetime'].dt.day
datetime_dim['pickup_month'] = datetime_dim['tpep_pickup_datetime'].dt.month
datetime_dim['pickup_year'] = datetime_dim['tpep_pickup_datetime'].dt.year
datetime_dim['pickup_weekday'] = datetime_dim['tpep_pickup_datetime'].dt.weekday
datetime_dim['tpep_pickup_datetime'] = datetime_dim['tpep_pickup_datetime'].dt.strftime('%Y-%m-%d')

datetime_dim['dropoff_time'] = datetime_dim['tpep_dropoff_datetime'].dt.strftime('%H:%M:%S')
datetime_dim['dropoff_day'] = datetime_dim['tpep_dropoff_datetime'].dt.day
datetime_dim['dropoff_month'] = datetime_dim['tpep_dropoff_datetime'].dt.month
datetime_dim['dropoff_year'] = datetime_dim['tpep_dropoff_datetime'].dt.year
datetime_dim['dropoff_weekday'] = datetime_dim['tpep_dropoff_datetime'].dt.weekday
datetime_dim['tpep_dropoff_datetime'] = datetime_dim['tpep_dropoff_datetime'].dt.strftime('%Y-%m-%d')

datetime_dim = datetime_dim[['datetime_id', 'tpep_pickup_datetime', 'pickup_time', 'pickup_day', 'pickup_month', 'pickup_year', 'pickup_weekday',
                             'tpep_dropoff_datetime','dropoff_time', 'dropoff_day', 'dropoff_month', 'dropoff_year', 'dropoff_weekday']]


datetime_dim.head()

In [ ]:
rate_code_type = {
1: 'Standard rate',
2:'JFK',
3:'Newark',
4:'Nassau or Westchester',
5:'Negotiated fare',
6:'Group ride'
}

rate_code_dim = trips[['RatecodeID']].reset_index(drop=True)
rate_code_dim['rate_code_id'] = rate_code_dim.index
rate_code_dim['rate_code_name'] = rate_code_dim['RatecodeID'].map(rate_code_types)
rate_code_dim = rate_code_dim[['rate_code_id', 'RatedcodeID', 'rate_code_name']]

rate_code_dim.head()

In [28]:
taxi_zones = pd.read_csv('data/taxi+_zone_lookup.csv')
taxi_zones.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [45]:
pickup_location_dim = trips[['PULocationID']].reset_index(drop=True)
pickup_location_dim.head()
pickup_location_dim = pickup_location_dim.merge(taxi_zones, how = 'left', left_on = 'PULocationID', right_on='LocationID')
pickup_location_dim['pickup_location_id'] = trips['PULocationID'].index

pickup_location_dim.rename(columns={'Borough': 'pu_borough', 'Zone': 'pu_zone', 'service_zone': 'pu_service_zone'}, inplace=True)
pickup_location_dim = pickup_location_dim[['pickup_location_id', 'pu_borough', 'pu_zone', 'pu_service_zone']]

In [46]:
pickup_location_dim.head()

,pickup_location_id,borough,zone,service_zone
0,0,Manhattan,Lincoln Square East,Yellow Zone
1,1,Manhattan,Upper East Side North,Yellow Zone
2,2,Manhattan,Morningside Heights,Boro Zone
3,3,Manhattan,Greenwich Village South,Yellow Zone
4,4,Manhattan,East Chelsea,Yellow Zone


In [48]:
pickup_location_dim.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39656098 entries, 0 to 39656097
Data columns (total 4 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   pickup_location_id  int64 
 1   borough             object
 2   zone                object
 3   service_zone        object
dtypes: int64(1), object(3)
memory usage: 1.5+ GB


In [51]:
dropoff_location_dim = trips[['DOLocationID']].reset_index(drop=True)
dropoff_location_dim.head()
dropoff_location_dim = dropoff_location_dim.merge(taxi_zones, how = 'left', left_on = 'DOLocationID', right_on='LocationID')
dropoff_location_dim['dropoff_location_id'] = trips['PULocationID'].index

dropoff_location_dim.rename(columns={'Borough': 'do_borough', 'Zone': 'do_zone', 'service_zone': 'do_service_zone'}, inplace=True)
dropoff_location_dim = dropoff_location_dim[['dropoff_location_id', 'do_borough', 'do_zone', 'do_service_zone']]

In [52]:
dropoff_location_dim.head()

,dropoff_location_id,borough,zone,service_zone
0,0,Manhattan,Upper East Side North,Yellow Zone
1,1,Manhattan,Central Harlem North,Boro Zone
2,2,Manhattan,Morningside Heights,Boro Zone
3,3,Manhattan,East Chelsea,Yellow Zone
4,4,Manhattan,Midtown North,Yellow Zone


In [53]:
dropoff_location_dim.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39656098 entries, 0 to 39656097
Data columns (total 4 columns):
 #   Column               Dtype 
---  ------               ----- 
 0   dropoff_location_id  int64 
 1   borough              object
 2   zone                 object
 3   service_zone         object
dtypes: int64(1), object(3)
memory usage: 1.5+ GB


In [20]:
payment_type_name = {
    1:'Credit card',
    2:'Cash',
    3:'No charge',
    4:'Dispute',
    5:'Unknown',
    6:'Voided trip'
}

payment_type_dim = trips[['payment_type']].reset_index(drop=True)
payment_type_dim['payment_type_id'] = payment_type_dim.index
payment_type_dim['payment_type_name'] = payment_type_dim['payment_type'].map(payment_type_name)
payment_type_dim = payment_type_dim[['payment_type_id', 'payment_type', 'payment_type_name']]

In [21]:
payment_type_dim.head()

,payment_type_id,payment_type,payment_type_name
0,0,1,Credit card
1,1,1,Credit card
2,2,1,Credit card
3,3,2,Cash
4,4,1,Credit card


In [22]:
payment_type_dim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39656098 entries, 0 to 39656097
Data columns (total 3 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   payment_type_id    int64 
 1   payment_type       int64 
 2   payment_type_name  object
dtypes: int64(2), object(1)
memory usage: 907.7+ MB


In [ ]:
fact_table = trips.merge(datetime_dim, left_on='trip_id', right_on='datetime_id') \
                .merge(rate_code_dim, left_on='trip_id', right_on='rate_code_id') \
                .merge(pickup_location_dim, left_on='trip_id', right_on='pickup_location_id') \
                .merge(dropoff_location_dim, left_on='trip_id', right_on='dropoff_location_id') \
                .merge(payment_type_dim, left_on='trip_id', right_on='payment_type_id') \
                .merge(rate_code_dim, left_on='trip_id', right_on='rate_code_id') \

In [23]:
fact_table = fact_table[[
    'trip_id',
    'datetime_id',
    'ratecode_id',
    'pickup_location_id',
    'dropout_location_id',
    'payment_type',
    'vendor_id',
    'store_and_fwd_flag',
    'passenger_count',
    'trip_distance',
    'fare_amount',
    'extra',
    'mta_tax',
    'tip_amount',
    'tolls_amount',
    'improvement_surcharge',
    'total_amount',
    'congestion_surcharge',
    'airport_fee
]]


Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee', 'trip_id'],
      dtype='object')

In [ ]:
fact_table.head()

In [ ]:
fact_table.tail()

In [ ]:
fact_table_info()